In [1]:
## downloading corpus

In [2]:
import keras
import numpy as np

2024-07-01 11:29:22.490388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 11:29:22.635760: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-01 11:29:22.635778: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-01 11:29:22.662136: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-01 11:29:23.544515: W tensorflow/stream_executor/platform/de

In [5]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)

In [6]:
text = open(path).read().lower()
print('Corpus length : ', len(text))

Corpus length :  600893


In [7]:
## Vectorization of sequences

In [8]:
maxlen = 60
step = 6
sentences = []
next_char = []

In [9]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_char.append(text[i + maxlen])

In [10]:
chars = sorted(list(set(text)))

In [13]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [18]:
x = np.zeros(shape = (len(sentences), maxlen, len(chars)))

In [19]:
y = np.zeros(shape = (len(next_char), len(chars)))

In [20]:
for i, sentence in enumerate(sentences):
    for j, ch in enumerate(chars):
        x[i, j, char_indices[ch]] = 1
    y[i, char_indices[next_char[i]]] = 1

In [23]:
from keras import layers, models, optimizers

In [25]:
model = models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = 'softmax'))

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               95232     
                                                                 
 dense_1 (Dense)             (None, 57)                7353      
                                                                 
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(
    optimizer = optimizers.RMSprop(learning_rate = 0.001),
    loss = 'categorical_crossentropy'
)

In [29]:
import random
import sys

In [30]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.randoom.multinomial(1, preds, 1)
    return np.argmax(probs)

In [31]:
for epoch in range(1, 60):
    print("Epoch : ", epoch)
    model.fit(x, y, batch_size = 128, epochs = 1)
    start_index = np.random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index : start_index + maxlen]
    print("-------------- Generated Text Seed------------------")
    print(generated_text)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f"----------------- temperature -------------{temperature}")
        sys.out.write(generated_text)
        for i in range(400):
            sample = np.zeros(shape = (1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sample[1, t, char_indices[char]] = 1
            preds = model.predict(sampled, verbose = 0)[0]
            next_index = sample(preds, temperature = temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

Epoch :  1


2024-07-01 12:19:30.187535: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1369901520 exceeds 10% of free system memory.


783/783 [==============================] - 66s 82ms/step - loss: 3.0372
-------------- Generated Text Seed------------------
attempt, it is commanded by the conscience of
logical method
----------------- temperature -------------0.2


AttributeError: module 'sys' has no attribute 'out'